In [1]:
import os
import pandas as pd

In [2]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

True

In [4]:
sql = """
   SELECT acc.id as `ID`,
       acc.creation_date as `CREACION`,
       acc.execution_date as `EJECUCION`,
       acc.concept as `CONCEPTO`,
       pr.name as `PROJECTO`,
       int.name as `INTERVENCION`,
       ph.name as `FASE`,
       CASE 
        WHEN acc.type = 'expense'  THEN 'Gasto'
        WHEN acc.type = 'income'  THEN 'Ingreso'
        WHEN acc.type = 'movement'  THEN 'Movimiento'
        ELSE ''
      END AS `TIPO`,
       acc.amount as `CANTIDAD`,
       acc.vat as `IVA`,
       acc.vat_amount as `IVA_CTA`,
       tg.name as `CAJA_BANCO`,
       ac.name as `CATEGORIA`,
       acc.description as `DESCRIPCION`,
       cr.name as `CREADOR`,
       rs.name as `RESPONSABLE`,      
       acc.code as `COTADOR`,
       acc.origin as `ORIGEN`,
       acc.place as `LUGAR`,
       acc.receipt as `FACTURA`
       
      
FROM `oan-nikarit.contoan.accounting` acc
LEFT OUTER JOIN `oan-nikarit.contoan.accounts` ac
ON acc.account_id = ac.id
LEFT OUTER JOIN `oan-nikarit.contoan.accounts` tg
ON acc.target_id = tg.id
LEFT OUTER JOIN `oan-nikarit.contoan.projects` pr
ON acc.project = pr.id
LEFT OUTER JOIN `oan-nikarit.contoan.projects` int
ON acc.intervention = int.id
LEFT OUTER JOIN `oan-nikarit.contoan.projects` ph
ON acc.phase = ph.id
LEFT OUTER JOIN `oan-nikarit.contoan.users` cr
ON acc.creator_user = cr.id
LEFT OUTER JOIN `oan-nikarit.contoan.users` rs
ON acc.user_in_charge = rs.id
""".lower()

# Run a Standard SQL query using the environment's default project
df = pd.read_gbq(sql)

Downloading: 100%|██████████| 2410/2410 [00:01<00:00, 1749.06rows/s]


In [83]:
df['ano_mes'] =  df['ejecucion'].dt.year.apply(str)+df['ejecucion'].dt.month.apply(lambda x: '0'+str(x) if x<10 else str(x))

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   id            2410 non-null   object             
 1   creacion      2410 non-null   datetime64[ns, UTC]
 2   ejecucion     2410 non-null   datetime64[ns, UTC]
 3   concepto      2410 non-null   object             
 4   projecto      2410 non-null   object             
 5   intervencion  2410 non-null   object             
 6   fase          784 non-null    object             
 7   tipo          2410 non-null   object             
 8   cantidad      2410 non-null   float64            
 9   iva           2406 non-null   float64            
 10  iva_cta       2405 non-null   float64            
 11  caja_banco    2410 non-null   object             
 12  categoria     2410 non-null   object             
 13  descripcion   2339 non-null   object             
 14  creador 

In [85]:
df[(df['factura'] != '') & 
   ~(df['factura'].isna())].shape

(342, 22)

In [86]:
df_cf = df[(df['factura'] != '') & 
   ~(df['factura'].isna())]

In [87]:
df_sf = df[~(df.index.isin(df_cf.index))]

In [88]:
df_sf['projecto'].unique()

array(['Nikarit', 'Administration_general', 'Emancipacion', 'ADSAN',
       'Aguapp', 'Évènements', 'Fassol', 'Forum Nikki', "Pompes d'eau"],
      dtype=object)

In [89]:
df_sf[(df_sf['creador'] == 'Mireille') &
      ((df_sf['projecto'] == 'Nikarit') |
       (df_sf['projecto'] == 'Emancipacion'))].groupby('ano_mes').count()['id']

ano_mes
201909     1
202002     1
202003     2
202004     3
202008     5
202009     7
202010    12
202011     6
202012     7
Name: id, dtype: int64

In [90]:
df_ob = df_sf[(df_sf['creador'] == 'Mireille') &
              (df_sf['origen'] == 'contoan') &
              (df_sf['tipo'] == 'gasto') &
              ((df_sf['projecto'] == 'Nikarit') |
               (df_sf['projecto'] == 'Emancipacion'))]

In [91]:
df_ob['lugar'].value_counts()

benin    38
Name: lugar, dtype: int64

In [96]:
d_t = df_ob.sort_values('ejecucion',ascending=False).T.to_dict()
for d in d_t:
    row = d_t[d]
    id_mov = row['id']
    concepto = row['concepto']
    e_date = str(row['ejecucion'])[0:10]
    url = 'https://nikarit.oaninternational.org/accounting/details/'+id_mov
    print(e_date,concepto,url,'\n')

2020-12-31 salaire mireill https://nikarit.oaninternational.org/accounting/details/JeBqTh8Qs8pdK5oor9CC 

2020-12-31 salaire animatrice https://nikarit.oaninternational.org/accounting/details/UuUjfIx2G2HhyzgI6V8d 

2020-12-15 transport pour rencontrer les impayés de crédit sianson https://nikarit.oaninternational.org/accounting/details/j5k2wcUzLXMzcjexZ5kG 

2020-12-06 15000 https://nikarit.oaninternational.org/accounting/details/tALE8iLqcCB3jI4Q1vvH 

2020-12-06 salaire younouss https://nikarit.oaninternational.org/accounting/details/LIoXWYcAZ4rpmNNP7i22 

2020-12-01 achat crédit https://nikarit.oaninternational.org/accounting/details/IMGLPH1C41d6d1qIFAxg 

2020-12-01 crédit pour appel   https://nikarit.oaninternational.org/accounting/details/m5EehMu4Ltco4WZ1ad6o 

2020-11-30 salaire Mireille https://nikarit.oaninternational.org/accounting/details/qCxezZFI7hM5cY1h4D9y 

2020-11-30 salaire coordonnatrice https://nikarit.oaninternational.org/accounting/details/UbhcExnhpq32d2rwNsSR 

202